# Lab. 2 Multi Layered Networks

### Ładowanie danych

PyTroch, a właściwie pakiet `torchvision` udostępnia parę przydatnych rzeczy, z których skorzystamy na dzisiejszych zajęciach.

Zacznijmy od ściąganie i ładowania danych, w [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html) znajdziemy popularne datasety, zajmiemy się dzisiaj MNISTem.

In [17]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms

train_data = MNIST(root='.', download=True, train=True, transform=transforms.ToTensor())
test_data = MNIST(root='.', download=True, train=False, transform=transforms.ToTensor())

train_data

mean = (train_data.train_data.type(torch.float32) / 255).mean().item()
std = (train_data.train_data.type(torch.float32) / 255).std().item()
print(mean, std)

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((mean, ), (std, )),
    transforms.Lambda(lambda t: t.view(784))
])


train_data = MNIST(root='.', download=True, train=True, transform=data_transform)
test_data = MNIST(root='.', download=True, train=False, transform=data_transform)

0.13054749369621277 0.30810782313346863


Oprócz tego z samego `torcha` możemy skorzystać z [`DataLoadera`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), który załatwia za nas sporo przydatnych rzeczy typu shufflowanie i batchowanie danych.

In [18]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_data, batch_size=10)

for x, y in train_loader:
    print(x.shape)
    print(x.dtype)
    print(y)
    break

torch.Size([10, 784])
torch.float32
tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4])


Wygląda na to, że aż tak bardzo za darmo wszystkiego nie dostaniemy, klasa `MNIST` zwraca nam dane w postaci obiektów [PILa](https://pillow.readthedocs.io/en/stable/). Musimy coś z tym zrobić.

## Zadanie 1.
1. Za pomocą [`transformerów`](https://pytorch.org/docs/stable/torchvision/transforms.html) przerobić powyższy kod tak aby zadziałał.  
**HINT**: sprawdzić jakie argumenty przyjmuje klasa `MNIST`.
2. Policzyć średnią i odchylenie standardowe wartości pojedynczego piksela dla całego zbioru trenującego i użyć ich do znormalizowania danych trenujących.  
**HINT**: Tutaj torchvision też powinien nam to ułatwić.
3. Zmienić "kształt" jednego przykładu z `28x28` na `784`.  
**HINT**: [`Lambda`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Lambda)

Uwaga: zwrócić uwagę co dokładnie robią używane _transformery_!

## Zadanie 2.

Ręcznie zaimplementować prostą sieć z jedną warstwą ukrtyą. Sieć ma mieć:
1. Jedną warstwę ukrytą rozmiaru 500 z wagami zainicjalizowanymi ze standardowego rozkładu normalnego.
2. Warstwa przy obu operacjach ma mieć uczone _biasy_ zainicjalizowane na 0.

**HINT**: Do rozkładu normalnego najlepiej użyć [`torch.randn`](https://pytorch.org/docs/stable/torch.html#torch.randn). Sprawdzić jakie ważne argumenty ta funkcja przyjmuje!

Należy oprócz tego zaimplementować pętlę uczenia z użyciem PyTorchowej funkcji kosztu _cross entropy_ i optymalizatora SGD.

In [37]:
from typing import List

class CustomNetwork(object):
    """
    Simple 1-hidden layer linear neural network
    """
    def __init__(self, input_size, activation, n_classes):
        """
        Initialize network's weights 
        """
        self.activation = activation
        self.n_classes = n_classes
        self.weight_1: torch.Tensor = torch.randn(input_size, 500, requires_grad=True)
        self.bias_1: torch.Tensor = torch.zeros(500, requires_grad=True)
        
        self.weight_2: torch.Tensor = torch.randn(500, n_classes, requires_grad=True)
        self.bias_2: torch.Tensor = torch.zeros(n_classes, requires_grad=True)
        
    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through the network
        """
        
        out_1 = torch.matmul(x, self.weight_1) + self.bias_1
        if self.activation:
            out_1 = self.activation(out_1)
        
        return torch.matmul(out_1, self.weight_2) + self.bias_2
    
    def parameters(self) -> List[torch.Tensor]:
        """
        Returns all trainable parameters 
        """
        return [self.weight_1, self.bias_1, self.weight_2, self.bias_2]

In [38]:
from torch.optim import SGD
from torch.nn.functional import cross_entropy, relu


# some hyperparams
batch_size: int = 64
epoch: int = 3
lr: float = 0.01
momentum: float = 0.9

# prepare data loaders, base don the already loaded datasets
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

input_size = 784
n_classes = 10

# initialize the model
model: CustomNetwork = CustomNetwork(input_size=input_size, activation=None, n_classes=n_classes)

# initialize the optimizer
optimizer: torch.optim.Optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

# training loop
for e in range(epoch):
    for i, (x, y) in enumerate(train_loader):
        
        # reset the gradients from previouis iteration
        optimizer.zero_grad()
        # pass through the network
        output: torch.Tensor = model(x)
        # calculate loss
        loss: torch.Tensor = cross_entropy(output, y)
        # backward pass thorught the network
        loss.backward()
        # apply the gradients
        optimizer.step()
        
        # log the loss value
        if (i + 1) % 100 == 0:
            print(f"Epoch {e} iter {i+1}/{len(train_data) // batch_size} loss: {loss.item()}", end="\r")
            
    # at the end of an epoch run evaluation on the test set
    with torch.no_grad():
        # initialize the number of correct predictions
        correct: int = 0 
        for i, (x, y) in enumerate(test_loader):
            # pass through the network
            output: torch.Tensor = model(x)
            # update the number of correctly predicted examples
            p = torch.argmax(output, dim=1)
            correct += torch.sum((p == y).float())

        print(f"\nTest accuracy: {correct / len(test_data)}")

        
# this is your test
assert correct / len(test_data) > 0.8, "Subject to random seed you should be able to get >80% accuracy"

Epoch 0 iter 900/937 loss: 45.882644653320318
Test accuracy: 0.7986999750137329
Epoch 1 iter 900/937 loss: 29.869281768798828
Test accuracy: 0.8266000151634216
Epoch 2 iter 900/937 loss: 18.753238677978516
Test accuracy: 0.8400999903678894


## Zadanie 3.

1. Przepisać całą sieć do PyTorcha używając [`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), [`torch.nn.Linear`](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear).
2. Dodać [nieliniowe aktywacje](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity) i dodatkową warstwę, tak aby wyciągnąć przynajmniej 95% testowego accuracy w 3 epoki.

In [47]:
class TorchNetwork(torch.nn.Module):
    """
    Simple 2-hidden layer non-linear neural network
    """
    
    def __init__(self, input_size, activation, n_classes):
        super(TorchNetwork, self).__init__()
        self.h1 = torch.nn.Linear(input_size, 500)
        self.h2 = torch.nn.Linear(500, 300)
        self.out = torch.nn.Linear(300, n_classes)
        self.act = activation
        
    def __call__(self, x):
        out_h1 = self.act(self.h1(x))
        out_h2 = self.act(self.h2(out_h1))
        return self.out(out_h2)


In [48]:
from torch.optim import SGD
from torch.nn.functional import cross_entropy

# some hyperparams
batch_size: int = 64
epoch: int = 3
lr: float = 0.01
momentum: float = 0.9

# prepare data loaders, base don the already loaded datasets
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

input_size = 784
n_classes = 10

# initialize the model
model: TorchNetwork = TorchNetwork(input_size=input_size, activation=relu, n_classes=n_classes)

# initialize the optimizer
optimizer: torch.optim.Optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

# training loop
for e in range(epoch):
    for i, (x, y) in enumerate(train_loader):
        
        # reset the gradients from previouis iteration
        optimizer.zero_grad()
        # pass through the network
        output: torch.Tensor = model(x)
        # calculate loss
        loss: torch.Tensor = cross_entropy(output, y)
        # backward pass thorught the network
        loss.backward()
        # apply the gradients
        optimizer.step()
        
        # log the loss value
        if (i + 1) % 100 == 0:
            print(f"Epoch {e} iter {i+1}/{len(train_data) // batch_size} loss: {loss.item()}", end="\r")
            
    # at the end of an epoch run evaluation on the test set
    with torch.no_grad():
        # initialize the number of correct predictions
        correct: int = 0 
        for i, (x, y) in enumerate(test_loader):
            # pass through the network
            output: torch.Tensor = model(x)
            # update the number of correctly predicted examples
            p = torch.argmax(output, dim=1)
            correct += torch.sum((p == y).float())

        print(f"\nTest accuracy: {correct / len(test_data)}")
            
            
# this is your test       
assert correct / len(test_data) > 0.95, "Subject to random seed you should be able to get >95% accuracy"

Epoch 0 iter 900/937 loss: 0.052613385021686554
Test accuracy: 0.9506000280380249
Epoch 1 iter 900/937 loss: 0.033850342035293585
Test accuracy: 0.9679999947547913
Epoch 2 iter 900/937 loss: 0.019509539008140564
Test accuracy: 0.9692999720573425
